In [22]:
#Read R1_link_main_list_df.csv file into df
import pandas as pd
import numpy as np

temp_df=pd.read_csv('C:\\Users\\Admin\\clean_link_main_list_df.csv')

temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      612 non-null    int64  
 1   Web_link        612 non-null    object 
 2   Property_Name   612 non-null    object 
 3   price           612 non-null    int64  
 4   no_of_bedrms    612 non-null    int64  
 5   no_of_bathrms   612 non-null    int64  
 6   Floor_Area      612 non-null    int64  
 7   Type            612 non-null    object 
 8   Age             612 non-null    float64
 9   log_price       612 non-null    float64
 10  log_Floor_Area  612 non-null    float64
dtypes: float64(3), int64(5), object(3)
memory usage: 52.7+ KB


In [23]:
# Data cleaning and 'Type' to make dummy data
temp_df = temp_df.replace('?', np.NaN).dropna().reset_index()
temp_df['price'] = temp_df['price'].astype(float)

homes = temp_df.select_dtypes(exclude=['object']).copy()
homes['Type'] = temp_df['Type']

In [24]:
#number of additional dummy column
homes['Type'].nunique()

11

In [25]:
# look at the dummies columns in detail
pd.get_dummies(homes['Type']).head(5)

,Apartment,Bungalow House,Cluster House,Condominium,Corner Terrace,Detached House,Executive Condominium,HDB,Semi-Detached House,Terraced House,Walk-up
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0


In [26]:
#combine the list togther with the main dataset
homes = pd.get_dummies(homes) 
homes.head(3)

,index,Unnamed: 0,price,no_of_bedrms,no_of_bathrms,Floor_Area,Age,log_price,log_Floor_Area,Type_Apartment,Type_Bungalow House,Type_Cluster House,Type_Condominium,Type_Corner Terrace,Type_Detached House,Type_Executive Condominium,Type_HDB,Type_Semi-Detached House,Type_Terraced House,Type_Walk-up
0,0,2,1550000.0,4,3,1528,7.0,14.253765,7.331715,0,0,0,1,0,0,0,0,0,0,0
1,1,3,450000.0,3,2,1001,5.0,13.017003,6.908755,0,0,0,0,0,0,0,1,0,0,0
2,2,4,3900000.0,5,5,4553,9.0,15.176487,8.423542,0,0,0,0,0,0,0,0,1,0,0


In [27]:
#Run simple Validation

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

X, y = homes.drop('price',axis=1), homes['price']

# Set final testing:@ 20% (test_size=.2)
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=8)

In [28]:
# Set final training @ 20% (test_size=.2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=88)

In [29]:
#set up the 3 model options

# LR Settings
lm = LinearRegression()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

#RR Settings
lm_reg = Ridge(alpha=1)

# Poly Settings
poly = PolynomialFeatures(degree=2) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

lm_poly = LinearRegression()

In [30]:
#validate using 3 models

#LR fit
lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

#RR fit
lm_reg.fit(X_train_scaled, y_train)
print(f'Ridge Regression val R^2: {lm_reg.score(X_val_scaled, y_val):.3f}')

#Poly fit Poly @ 2 Deg
lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

Linear Regression val R^2: 0.892
Ridge Regression val R^2: 0.893
Degree 2 polynomial regression val R^2: -5.510


In [36]:
#Poly fit Poly @ 1 Deg
poly = PolynomialFeatures(degree=1) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

lm_poly = LinearRegression()
lm_poly.fit(X_train_poly, y_train)
print(f'REtry Degree 1 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

REtry Degree 1 polynomial regression val R^2: 0.892


In [32]:
# Find out test scores
lm.fit(X,y)
print(f'Linear Regression test R^2: {lm.score(X_test, y_test):.3f}')

lm_reg.fit(X,y)
print(f'Ridge Regression test R^: {lm_reg.score(X_test, y_test):.3f}')

Linear Regression test R^2: 0.661
Ridge Regression test R^: 0.655


In [17]:
#Run rigorous validation
from sklearn.model_selection import KFold

X, y = homes.drop('price',axis=1), homes['price']

#Set final testing:@ 20% (test_size=.2)
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10)

#for kf to generate indices
X, y = np.array(X), np.array(y)

In [18]:
#Start CV

kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] 
# results for LR and RR

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

Simple regression scores:  [0.8293050323118546, 0.5943014021678077, 0.8004051661790965, 0.8451196988486388, 0.6416752099848932]
Ridge scores:  [0.8109535579305298, 0.6130057426422182, 0.8136114637706705, 0.8489651051833784, 0.6355203979673596] 

Simple mean cv r^2: 0.742 +- 0.103
Ridge mean cv r^2: 0.744 +- 0.099


In [19]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

lm = LinearRegression()
lm.fit(X_scaled,y)
print(f'Linear Regression test R^2: {lm.score(X_test_scaled, y_test):.3f}')

lm_reg = Ridge(alpha=1)
lm_reg.fit(X_scaled,y)
print(f'Ridge Regression test R^2: {lm_reg.score(X_test_scaled, y_test):.3f}')

Linear Regression test R^2: 0.778
Ridge Regression test R^2: 0.787
